In [844]:
import pandas as pd
import numpy as np

marketValue_df = pd.read_csv("https://storm.cis.fordham.edu/~jcao42/2010-2025.csv")

In [845]:
marketValue_df.head()

,date,club_name,market_value,current_value
0,2010-11-01,Manchester City,€360.30m,€1.31bn
1,2010-11-01,Arsenal FC,€303.00m,€1.13bn
2,2010-11-01,Liverpool FC,€275.00m,€993.50m
3,2010-11-01,Chelsea FC,€360.00m,€922.00m
4,2010-11-01,Tottenham Hotspur,€282.70m,€836.10m


#processing data for marketValue data sets


In [847]:
marketValue_df.info() #overlook of 2010-2025 market value datasets

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6900 entries, 0 to 6899
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           6900 non-null   object
 1   club_name      6900 non-null   object
 2   market_value   6900 non-null   object
 3   current_value  6900 non-null   object
dtypes: object(4)
memory usage: 215.8+ KB


In [848]:
parsed_df = marketValue_df.copy()

In [849]:
parsed_df["date"] = pd.to_datetime(marketValue_df["date"])
#define a function to translate money symbol to numerical value
def parse_value(value_str):
    s = value_str.strip().replace("€","")
    if s == "-":
        return np.nan
    if s.endswith("bn"):
        numeric_part = s.replace("bn", "")
        return float(numeric_part)*1e9
    elif s.endswith("m"):
        numeric_part = s.replace("m","")
        return float(numeric_part) * 1e6
    else:
        return float(s)
parsed_df["market_value"] = marketValue_df["market_value"].apply(parse_value)
parsed_df["current_value"] = marketValue_df["current_value"].apply(parse_value)

In [850]:
marketValue_df.head()

,date,club_name,market_value,current_value
0,2010-11-01,Manchester City,€360.30m,€1.31bn
1,2010-11-01,Arsenal FC,€303.00m,€1.13bn
2,2010-11-01,Liverpool FC,€275.00m,€993.50m
3,2010-11-01,Chelsea FC,€360.00m,€922.00m
4,2010-11-01,Tottenham Hotspur,€282.70m,€836.10m


In [851]:
parsed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6900 entries, 0 to 6899
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           6900 non-null   datetime64[ns]
 1   club_name      6900 non-null   object        
 2   market_value   6680 non-null   float64       
 3   current_value  6900 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 215.8+ KB


In [852]:
parsed_df.isna().sum() #check missing value

date               0
club_name          0
market_value     220
current_value      0
dtype: int64

In [853]:
#since the data update every 15 days, we consider it is relatively short. So, use data from 15 days ago and 1 month ago will be a greate way to replace missing value.
parsed_df = parsed_df.sort_values(["club_name", "date"]).reset_index(drop=True)
parsed_df["market_value"] = parsed_df.groupby("club_name")["market_value"].ffill()

In [854]:
parsed_df.isna().sum() #check missing value, now it should be 0

date             0
club_name        0
market_value     0
current_value    0
dtype: int64

In [855]:
RawData_df = pd.read_csv("https://storm.cis.fordham.edu/~jcao42/RawDataPremier.csv")

In [856]:
RawData_df.tail()

,Team_ID,TeamName,Season,Wins,Losses,Goals,Goal_Conceded,GoalsPerMatch,Shots,ShotsOnTarget,ShootingAccuracy%,PenaltiesScored,BigChancesCreated,HitWoodwork,YellowCards,RedCards,Fouls,Offsides,CleanSheets,GoalsConcededPerMatch,Saves,Tackle,TackleSuccess%,BlockedShots,Interceptions,Clearances,HeadedClearance,AerialBattle/DuelsWon,ErrorsLeadingToGoal,OwnGoals,Champion,FiredManager
193,15,Nottingham-Forest,2023/24,9,20,49,67,1.29,451,148,33%,1,65,15,78,3,311,77,4,1.76,103,728,59%,116,331,910,440,"2,554",8,2,0,1
194,18,Sheffield-United,2023/24,3,28,35,104,0.92,365,129,35%,5,40,7,95,5,306,64,1,2.74,170,709,57%,105,347,997,451,"2,466",9,7,0,1
195,21,Tottenham-Hotspur,2023/24,20,12,74,61,1.95,584,215,37%,2,90,18,89,4,314,83,7,1.61,110,724,62%,171,355,652,334,"2,319",6,3,0,0
196,25,West-Ham-United,2023/24,14,14,60,74,1.58,450,154,34%,4,58,14,76,3,300,71,5,1.95,171,684,62%,112,361,862,417,"2,585",3,4,0,0
197,38,Wolverhampton-Wanderers,2023/24,13,18,50,65,1.32,429,160,37%,4,54,12,100,4,340,78,5,1.71,142,739,58%,111,284,786,406,"2,450",3,1,0,0


In [857]:
manager_Changes_df = pd.read_csv("https://storm.cis.fordham.edu/~jcao42/manager_changes_2010_2025.csv")

In [858]:
manager_Changes_df.head()

,season,change_type,club,coach_name,role,matchday,rank1,ppg1,last_match_home,last_match_away,last_match_score,leaving_date,days_in_charge,successor_name,successor_role,rank2,ppg2,first_match_home,first_match_away,first_match_score
0,2010,Change(s) in coach at the end of the season,Chelsea FC,Carlo Ancelotti,Manager,38,2,1.87,Everton FC,Chelsea FC,1:0,"Jun 30, 2011",729,André Villas-Boas,Manager,-,-,NaN,NaN,NaN
1,2010,Change(s) in coach at the end of the season,Birmingham City,Alex McLeish,Manager,38,18,1.03,Tottenham Hotspur,Birmingham City,2:1,"Jun 12, 2011",1293,Chris Hughton,Manager,-,-,NaN,NaN,NaN
2,2010,Change(s) in coach at the end of the season,Fulham FC,Mark Hughes,Manager,38,8,1.29,Fulham FC,Arsenal FC,2:2,"Jun 2, 2011",308,Martin Jol,Manager,-,-,NaN,NaN,NaN
3,2010,Change(s) in coach at the end of the season,Aston Villa,Gérard Houllier,Manager,38,9,1.21,Aston Villa,Liverpool FC,1:0,"Jun 1, 2011",265,Alex McLeish,Manager,-,-,NaN,NaN,NaN
4,2010,Change(s) in coach at the end of the season,West Ham United,Kevin Keen,Caretaker Manager,38,20,0.00,West Ham United,Sunderland AFC,0:3,"May 31, 2011",16,Sam Allardyce,Manager,-,-,NaN,NaN,NaN


In [859]:
manager_Changes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   season             248 non-null    int64 
 1   change_type        248 non-null    object
 2   club               248 non-null    object
 3   coach_name         248 non-null    object
 4   role               248 non-null    object
 5   matchday           248 non-null    object
 6   rank1              248 non-null    object
 7   ppg1               248 non-null    object
 8   last_match_home    231 non-null    object
 9   last_match_away    231 non-null    object
 10  last_match_score   231 non-null    object
 11  leaving_date       248 non-null    object
 12  days_in_charge     248 non-null    int64 
 13  successor_name     248 non-null    object
 14  successor_role     248 non-null    object
 15  rank2              248 non-null    object
 16  ppg2               248 non-null    object
 1

In [860]:
#in the role, caretaker manager will not be considered since it not benefits for our analysis.
manager_Changes_df = manager_Changes_df[~manager_Changes_df["role"].str.lower().str.contains("caretaker")]
#change data as datetime
manager_Changes_df["leaving_date"] = pd.to_datetime(manager_Changes_df["leaving_date"])

In [861]:
manager_Changes_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 247
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   season             171 non-null    int64         
 1   change_type        171 non-null    object        
 2   club               171 non-null    object        
 3   coach_name         171 non-null    object        
 4   role               171 non-null    object        
 5   matchday           171 non-null    object        
 6   rank1              171 non-null    object        
 7   ppg1               171 non-null    object        
 8   last_match_home    164 non-null    object        
 9   last_match_away    164 non-null    object        
 10  last_match_score   164 non-null    object        
 11  leaving_date       171 non-null    datetime64[ns]
 12  days_in_charge     171 non-null    int64         
 13  successor_name     171 non-null    object        
 14  successor_role 

In [862]:
#check if they been fired
def determine_fired(change_type):
    if pd.isna(change_type):
        return np.nan
    change_type = change_type.lower()
    if "during the season" in change_type:
        return 1
    elif "at the end of the season" in change_type:
        return 0
    else:
        return np.nan  

manager_Changes_df["fired"] = manager_Changes_df["change_type"].apply(determine_fired)

In [863]:
#df after cleaning 
manager_clean = manager_Changes_df[[
    "season", "club", "coach_name", "leaving_date", "days_in_charge", "fired"
]].rename(columns={
    "club": "club_name",
    "coach_name": "manager_name"
})


In [864]:
manager_clean.head()

,season,club_name,manager_name,leaving_date,days_in_charge,fired
0,2010,Chelsea FC,Carlo Ancelotti,2011-06-30,729,0.0
1,2010,Birmingham City,Alex McLeish,2011-06-12,1293,0.0
2,2010,Fulham FC,Mark Hughes,2011-06-02,308,0.0
3,2010,Aston Villa,Gérard Houllier,2011-06-01,265,0.0
5,2010,West Ham United,Avram Grant,2011-05-15,318,1.0


In [865]:
manager_clean["fired"].value_counts(dropna=False)

fired
1.0    111
0.0     54
NaN      6
Name: count, dtype: int64

In [866]:
#deal with NaN value
#check record
print(manager_clean[manager_clean["fired"].isna()])

# what if we delete it
manager_clean = manager_clean.dropna(subset=["fired"])
print(manager_clean["fired"].value_counts(dropna=False))


     season                club_name     manager_name leaving_date  \
15     2010              Aston Villa   Martin O'Neill   2010-08-09   
76     2014           Crystal Palace       Tony Pulis   2014-08-14   
108    2016                Hull City      Steve Bruce   2016-07-22   
109    2016           Sunderland AFC    Sam Allardyce   2016-07-21   
144    2018               Chelsea FC    Antonio Conte   2018-07-13   
227    2023  Wolverhampton Wanderers  Julen Lopetegui   2023-08-08   

     days_in_charge  fired  
15             1466    NaN  
76              263    NaN  
108            1482    NaN  
109             286    NaN  
144             742    NaN  
227             267    NaN  
fired
1.0    111
0.0     54
Name: count, dtype: int64


In [867]:
#we can put some values by our self base on real situation. 
fired_manual = {
    "Martin O'Neill": 0,
    "Tony Pulis": 1,
    "Steve Bruce": 0,
    "Sam Allardyce": 0,
    "Antonio Conte": 1,
    "Julen Lopetegui": 0
}

manager_clean["fired"] = manager_clean.apply(
    lambda row: fired_manual[row["manager_name"]] if pd.isna(row["fired"]) and row["manager_name"] in fired_manual else row["fired"],
    axis=1
)

In [868]:
print(manager_clean["fired"].value_counts())

fired
1.0    111
0.0     54
Name: count, dtype: int64


In [869]:
print(manager_clean[["club_name", "season"]].drop_duplicates().head())
print(parsed_df[["club_name", "date"]].drop_duplicates().head())

         club_name  season
0       Chelsea FC    2010
1  Birmingham City    2010
2        Fulham FC    2010
3      Aston Villa    2010
5  West Ham United    2010
         club_name       date
0  AFC Bournemouth 2010-11-01
1  AFC Bournemouth 2010-11-15
2  AFC Bournemouth 2010-12-01
3  AFC Bournemouth 2010-12-15
4  AFC Bournemouth 2011-01-01


In [870]:
Income_Expanditures_df= pd.read_csv("https://storm.cis.fordham.edu/~jcao42/PremierLeague_Transfers_Income_expenditures_2014_2024.csv")

In [871]:
Income_Expanditures_df.head()

,year,club,expenditure,income,balance
0,2014,Manchester United,€195.35m,€42.90m,€-152.45m
1,2014,Liverpool FC,€151.43m,€91.97m,€-59.46m
2,2014,Chelsea FC,€106.70m,€96.51m,€-10.19m
3,2014,Arsenal FC,€101.73m,€25.20m,€-76.53m
4,2014,Southampton FC,€82.20m,€119.83m,€37.63m


In [872]:
Income_Expanditures_df.tail()

,year,club,expenditure,income,balance
215,2024,Wolverhampton Wanderers,€74.40m,€112.50m,€38.10m
216,2024,Newcastle United,€68.20m,€76.20m,€8.00m
217,2024,Everton FC,€50.20m,€83.65m,€33.45m
218,2024,Liverpool FC,€42.00m,€47.00m,€5.00m
219,2024,Manchester City,€25.00m,€141.00m,€116.00m


In [873]:
Income_Expanditures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   year         220 non-null    int64 
 1   club         220 non-null    object
 2   expenditure  220 non-null    object
 3   income       220 non-null    object
 4   balance      220 non-null    object
dtypes: int64(1), object(4)
memory usage: 8.7+ KB


In [874]:
marketValue_df[marketValue_df['club_name'] == 'Manchester City']

,date,club_name,market_value,current_value
0,2010-11-01,Manchester City,€360.30m,€1.31bn
20,2010-11-15,Manchester City,€357.30m,€1.31bn
40,2010-12-01,Manchester City,€357.30m,€1.31bn
60,2010-12-15,Manchester City,€357.30m,€1.31bn
80,2011-01-01,Manchester City,€357.30m,€1.31bn
100,2011-01-15,Manchester City,€374.30m,€1.31bn
120,2011-02-01,Manchester City,€351.30m,€1.31bn
140,2011-02-15,Manchester City,€366.75m,€1.31bn
160,2011-03-01,Manchester City,€366.75m,€1.31bn
180,2011-03-15,Manchester City,€366.75m,€1.31bn


In [875]:
RawData_df['Season'] = RawData_df['Season'].str.slice(0, 4)
RawData_df['TeamName'] = RawData_df['TeamName'].str.replace('-',' ')
manager_clean['club_name'] = manager_clean['club_name'].str.replace(' FC', '').str.replace(' LC', '').str.replace(' AFC', '').str.replace('&', 'and').str.replace('AFC ', '')

In [876]:
RawData_df = RawData_df.rename(columns={
    "TeamName": "club_name",
    "Season": "season"
})

In [877]:
RawData_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Team_ID                198 non-null    int64  
 1   club_name              198 non-null    object 
 2   season                 198 non-null    object 
 3   Wins                   198 non-null    int64  
 4   Losses                 198 non-null    int64  
 5   Goals                  198 non-null    int64  
 6   Goal_Conceded          198 non-null    int64  
 7   GoalsPerMatch          198 non-null    float64
 8   Shots                  198 non-null    int64  
 9   ShotsOnTarget          198 non-null    int64  
 10  ShootingAccuracy%      198 non-null    object 
 11  PenaltiesScored        198 non-null    int64  
 12  BigChancesCreated      198 non-null    int64  
 13  HitWoodwork            198 non-null    int64  
 14  YellowCards            198 non-null    int64  
 15  RedCar

In [878]:
RawData_df['season'] = RawData_df['season'].astype('int64')

In [879]:
manager_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, 0 to 247
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   season          165 non-null    int64         
 1   club_name       165 non-null    object        
 2   manager_name    165 non-null    object        
 3   leaving_date    165 non-null    datetime64[ns]
 4   days_in_charge  165 non-null    int64         
 5   fired           165 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 9.0+ KB


In [880]:
RawData_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Team_ID                198 non-null    int64  
 1   club_name              198 non-null    object 
 2   season                 198 non-null    int64  
 3   Wins                   198 non-null    int64  
 4   Losses                 198 non-null    int64  
 5   Goals                  198 non-null    int64  
 6   Goal_Conceded          198 non-null    int64  
 7   GoalsPerMatch          198 non-null    float64
 8   Shots                  198 non-null    int64  
 9   ShotsOnTarget          198 non-null    int64  
 10  ShootingAccuracy%      198 non-null    object 
 11  PenaltiesScored        198 non-null    int64  
 12  BigChancesCreated      198 non-null    int64  
 13  HitWoodwork            198 non-null    int64  
 14  YellowCards            198 non-null    int64  
 15  RedCar

In [881]:
df_merge = pd.merge(RawData_df, manager_clean, on=['club_name', 'season'], how='left')

In [882]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Team_ID                221 non-null    int64         
 1   club_name              221 non-null    object        
 2   season                 221 non-null    int64         
 3   Wins                   221 non-null    int64         
 4   Losses                 221 non-null    int64         
 5   Goals                  221 non-null    int64         
 6   Goal_Conceded          221 non-null    int64         
 7   GoalsPerMatch          221 non-null    float64       
 8   Shots                  221 non-null    int64         
 9   ShotsOnTarget          221 non-null    int64         
 10  ShootingAccuracy%      221 non-null    object        
 11  PenaltiesScored        221 non-null    int64         
 12  BigChancesCreated      221 non-null    int64         
 13  HitWo

In [887]:
#In RawData_df but not manager_clean: Brentford, Luton Town
#In manager_clean but not RawData_df: Queens Park Rangers, Wigan Athletic, Blackburn Rovers